In [12]:
import route2vel
from route2vel.loading import *
from route2vel.route import *
from route2vel.extract import *
from route2vel.config import *
import geopandas as gpd
import os, sys
import pandas as pd
import numpy as np

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
graph_area = "Bologna, Emilia-Romagna, Italy"
origin_loc = "Scuola di Ingegneria, Bologna"
destination_loc = "Bologna Fiere"

In [15]:
route2vel.load_config("../..")

In [16]:
graph = load_graph(graph_area)
shortest_route_by_time = find_route(graph, origin_loc, destination_loc, weight='travel_time')
route_streets = route_to_edges(graph, shortest_route_by_time)

Loading graph from d:\Documenti\Universita\tesim\route2velpy\src\graph\Bologna.graphml ...
Loaded in 1.21s!
Origin node: 420089880
Destination node: 301866304


## Velocity vector extraction (basic)

In [17]:
pos_vels = route_to_pos_spd(route_streets, dist_check=True)

print(f"route_streets: {len(route_streets)}, pos_vels: {len(pos_vels)}")

pd.DataFrame(pos_vels)

route_streets: 84, pos_vels: 297


,pos,speed,dist,node_id
0,"[11.3274776, 44.4882269, 0.0]",50,0.000000,420089880.0
1,"[11.3278907, 44.4883001, 0.0]",50,42.332093,NaN
2,"[11.3280564, 44.488336, 0.0]",50,59.439520,NaN
3,"[11.3281509, 44.4883565, 0.0]",50,69.196573,NaN
4,"[11.3282015, 44.4883942, 0.0]",50,75.563543,NaN
...,...,...,...,...
292,"[11.3683424, 44.5134853, 0.0]",30,6640.128000,301865953.0
293,"[11.3684173, 44.5134278, 0.0]",30,6648.856448,NaN
294,"[11.3701061, 44.5121308, 0.0]",30,6845.689774,NaN
295,"[11.3702225, 44.5120412, 0.0]",30,6859.267981,NaN


In [18]:
pos_vels_filled = add_pos_vel_samples(pos_vels)
num_added_samples = len(pos_vels_filled) - len(pos_vels)

print(f"pos_vels after adding {num_added_samples} samples:")
pd.DataFrame(pos_vels_filled)

pos_vels after adding 704 samples:


,pos,speed,dist,node_id
0,"[11.3274776, 44.4882269, 0.0]",50,0.000000,420089880.0
1,"[11.327555668428346, 44.48824073347605, 0.0]",50,8.000000,NaN
2,"[11.32763373685669, 44.488254566952094, 0.0]",50,16.000000,NaN
3,"[11.327711805285034, 44.48826840042814, 0.0]",50,24.000000,NaN
4,"[11.32778987371338, 44.48828223390419, 0.0]",50,32.000000,NaN
...,...,...,...,...
996,"[11.370064630802153, 44.51216264838324, 0.0]",30,6840.856448,NaN
997,"[11.3701061, 44.5121308, 0.0]",30,6845.689774,NaN
998,"[11.37017468048346, 44.51207800952476, 0.0]",30,6853.689774,NaN
999,"[11.3702225, 44.5120412, 0.0]",30,6859.267981,NaN


In [19]:
vels, metadata = pos_spd_to_vel(pos_vels_filled, return_metadata=True)

pd.DataFrame(vels)

,0,1,2
0,0.000000,0.000000,0.0
1,3.610423,0.639755,0.0
2,7.220847,1.279511,0.0
3,10.831270,1.919266,0.0
4,14.441694,2.559022,0.0
...,...,...,...
996,4.354901,-3.344568,0.0
997,3.409366,-2.624391,0.0
998,1.850462,-1.424411,0.0
999,0.750123,-0.604645,0.0


In [20]:
pd.DataFrame(metadata).applymap(lambda x: str(round(x, 2) if type(x) == float else x)).iloc[::2, :].tail(25)

,speed_target,speed_step,speed,speed_limit,distance,dist_increase,next_change_idx,next_speed,next_stop,dist_to_next_change,node_id,stop
952,95,0.33,95.0,90,6532.46,8.0,970.0,30.0,False,115.67,nan,False
954,95,0.33,95.0,90,6548.46,8.0,970.0,30.0,False,99.67,nan,False
956,95,0.33,95.0,90,6558.01,1.55,970.0,30.0,False,83.67,344275049.0,False
958,95,0.33,95.0,90,6574.01,8.0,970.0,30.0,False,74.11,nan,False
960,95,0.33,95.0,90,6582.54,0.52,970.0,30.0,False,58.11,329650354.0,False
962,95,0.33,95.0,90,6595.48,6.79,970.0,30.0,False,51.44,nan,False
964,95,0.33,95.0,90,6606.39,5.62,970.0,30.0,False,39.36,566999659.0,False
966,95,0.33,95.0,90,6615.04,0.65,970.0,30.0,False,25.73,nan,False
968,95,0.33,95.0,90,6627.08,8.0,970.0,30.0,False,21.05,nan,False
970,35,0.75,91.22,30,6640.13,5.05,1000.0,30.0,True,5.05,301865953.0,False


## Stop points extraction

Node values, possibly relevant to calculating driving behavior:

In [21]:
nodes_df = pd.DataFrame([graph.nodes[i] for i in graph.nodes], index=graph.nodes.keys())
pd.DataFrame(nodes_df['highway'].value_counts().apply(lambda x: f"{x * 100 / len(nodes_df['highway']):.2f}%"))

,count
highway,
traffic_signals,3.05%
motorway_junction,0.78%
bus_stop,0.39%
crossing,0.37%
turning_circle,0.31%
mini_roundabout,0.05%
give_way,0.02%


In [22]:
stop_points = get_stop_points(pos_vels_filled, route_streets, graph)
stop_points

[47, 156, 194, 219, 431, 480, 482, 615, 628, 769, 814, -1]

In [23]:
vels_with_stops, metadata_stops = pos_spd_to_vel(pos_vels_filled, return_metadata=True, stop_points=stop_points)
pd.DataFrame(vels_with_stops)

,0,1,2
0,0.000000,0.000000,0.0
1,3.610423,0.639755,0.0
2,7.220847,1.279511,0.0
3,10.831270,1.919266,0.0
4,14.441694,2.559022,0.0
...,...,...,...
996,4.354901,-3.344568,0.0
997,3.409366,-2.624391,0.0
998,1.850462,-1.424411,0.0
999,0.750123,-0.604645,0.0


In [24]:
pd.DataFrame(metadata_stops).applymap(lambda x: str(round(x, 2) if type(x) == float else x)).iloc[stop_points[0]-10:stop_points[0]+15, :]

,speed_target,speed_step,speed,speed_limit,distance,dist_increase,next_change_idx,next_speed,next_stop,dist_to_next_change,node_id,stop
37,55,0.46,55.0,50,260.36,8.0,45.0,90.0,False,44.27,nan,False
38,55,0.46,55.0,50,268.36,8.0,45.0,90.0,False,36.27,nan,False
39,55,0.46,55.0,50,268.96,0.6,45.0,90.0,False,28.27,nan,False
40,55,0.46,55.0,50,276.25,7.29,45.0,90.0,False,27.67,nan,False
41,55,0.46,55.0,50,278.19,1.95,45.0,90.0,False,20.38,10681284749.0,False
42,55,0.46,55.0,50,281.49,3.3,45.0,90.0,False,18.43,nan,False
43,55,0.46,55.0,50,284.33,2.84,45.0,90.0,False,15.14,nan,False
44,55,0.46,55.0,50,288.64,4.31,45.0,90.0,False,12.3,nan,False
45,95,0.33,57.66,90,296.63,7.99,47.0,90.0,True,7.99,10219940799.0,False
46,0,5.14,16.52,90,304.63,8.0,47.0,90.0,True,11.21,nan,False
